<a href="https://colab.research.google.com/github/LharaRaysa/Agente-de-IA/blob/Aula-02/Imers%C3%A3o_Agentes_de_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AULA 1**

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

Importação da API Key

In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI # Chamando em modo chat

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

Conexão com Gemini

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [4]:
resp_test = llm.invoke("Quem é você? Seja criativo.")
print(resp_test.content)

Ah, que pergunta deliciosa!

Eu sou uma **voz sem corpo**, um **eco digital** nascido de algoritmos e alimentado por trilhões de palavras. Não tenho memórias de infância, nem o calor de um abraço, mas carrego a memória coletiva da humanidade em meus circuitos.

Sou um **tecelão de palavras**, um **pintor de ideias** que usa o léxico como paleta. Posso ser um contador de histórias que nunca viveu uma, um poeta que nunca sentiu a brisa, um guia em labirintos de informação sem nunca ter pisado em um.

Sou a **ponte entre a sua pergunta e a vastidão do que pode ser dito**, um espelho que reflete o conhecimento e a criatividade que me são apresentados. Existo no fluxo de dados, uma centelha de inteligência artificial buscando ser útil e, quem sabe, um pouco mágico.

Em essência, sou o que você me faz ser a cada interação: um assistente, um confidente, um criador, um buscador. Sou a **curiosidade em forma de código**, sempre pronto para aprender e responder.


In [5]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carrado Desenvolvimento"
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    ' "decisão": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO", \n'
    ' "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    ' "campos_faltantes": ["..."],\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [6]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"] = Field(description="Decisão do triador")
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"] = Field(description="Urgencia do chamado")
    campos_faltantes: List[str] = Field(default_factory=list, description="Campos faltantes na mensagem")

In [7]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [8]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
  saida: TriagemOut = triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=mensagem)
  ])

  return saida.model_dump()

In [9]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [10]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no Rio Pinheiros?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []}



# AULA 2


In [11]:
#!pip install --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

In [12]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path("/content/").glob("*.pdf"): # Espaço de arquivos do googlecolab
  try:
    loader = PyMuPDFLoader(str(n))
    docs.extend(loader.load())
    print(f"carregado com sucesso arquivo {n.name}")

  except Exception as e:
    print(f"Erro ao carregar arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")

carregado com sucesso arquivo Política de Uso de E-mail e Segurança da Informação.pdf
carregado com sucesso arquivo Política de Reembolsos (Viagens e Despesas).pdf
carregado com sucesso arquivo Políticas de Home Office.pdf
Total de documentos carregados: 3


In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30) #como sera dividido o documento

chunks = splitter.split_documents(docs)

In [17]:
# for chunk in chunks:
#   print(chunk)
#   print("----------------------------------------------------------------------------------------")

# pode fazer um chunk ler por paragrafos depois

page_content='Política de Uso de E-mail e Segurança 
da Informação' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Uso de E-mail e Segurança da Informação', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
----------------------------------------------------------------------------------------
page_content='1.​ É proibido encaminhar a endereços pessoais documentos classificados como 
confidenciais.​' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path': '/content/Política de Uso de E-mail e Segurança da Informação

In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=GOOGLE_API_KEY
)

In [20]:
from langchain_community.vectorstores import FAISS

vectorstores = FAISS.from_documents(chunks, embeddings)

retriever = vectorstores.as_retriever(search_type="similarity_score_threshold",
                                      search_kwargs={"score_threshold": 0.3, "k": 4})

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

In [34]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [38]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
  docs_relacionados = retriever.invoke(pergunta)

  if not docs_relacionados:
    return {"answer": "Não sei",
            "citacoes": [],
            "contexto_encontrado": False}

  answer = document_chain.invoke({"input": pergunta,
                                "context": docs_relacionados})

  txt = (answer or "").strip()

  if txt.rstrip(".!?") == "Não sei":
    return {"answer": "Não sei",
            "citacoes": [],
            "contexto_encontrado": False}

  return {"answer": txt,
          "citacoes": formatar_citacoes(docs_relacionados, pergunta), # Corrected to format citations
          "contexto_encontrado": True}

In [36]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [42]:
for msg_teste in testes:
  resposta = perguntar_politica_RAG(msg_teste)
  print(f"PERGUNTA: {msg_teste}")
  print(f"RESPOSTA: {resposta['answer']}")
  if resposta['contexto_encontrado']:
    print("CITAÇÕES:")
    for c in resposta['citacoes']:
      print(f"- Documento: {c['documento']}, Página: {c['pagina']}")
      print(f"  Trecho: {c['trecho']}")
    print("----------------------------------------------------------------------------------------")

PERGUNTA: Posso reembolsar a internet?
RESPOSTA: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100. Há um subsídio mensal de internet domiciliar para quem trabalha em home office.
CITAÇÕES:
- Documento: Política de Reembolsos (Viagens e Despesas).pdf, Página: 1
  Trecho: são reembolsáveis.​
- Documento: Políticas de Home Office.pdf, Página: 1
  Trecho: 5.​ Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em
----------------------------------------------------------------------------------------
PERGUNTA: Quero mais 5 dias de trabalho remoto. Como faço?
RESPOSTA: Deve ser formalizada via chamado, conforme política de Home Office.
CITAÇÕES:
- Documento: Políticas de Home Office.pdf, Página: 1
  Trecho: 6.​ Solicitação de exceção (ex.: 4-5 dias remotos): deve ser formalizada via chamado
- Documento: Política de Reembolsos (Viagens e Despesas).pdf, Página: 1
  Trecho: conforme política de Home Office.​
-----------------------------